In [50]:
from bokeh.io import curdoc, output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
import pandas as pd
from bokeh.layouts import row, widgetbox
from bokeh.models import Slider, Select

In [20]:
data=pd.read_csv('Desktop/Python/gapminder.csv')

FileNotFoundError: File b'Desktop/Python/gapminder.csv' does not exist

In [27]:
data.head()

,Country,Year,fertility,life,population,child_mortality,gdp,region
0,Afghanistan,1964,7.671,33.639,10474903.0,339.7,1182.0,South Asia
1,Afghanistan,1965,7.671,34.152,10697983.0,334.1,1182.0,South Asia
2,Afghanistan,1966,7.671,34.662,10927724.0,328.7,1168.0,South Asia
3,Afghanistan,1967,7.671,35.170,11163656.0,323.3,1173.0,South Asia
4,Afghanistan,1968,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [33]:
source=ColumnDataSource(data={'x': data.loc[data['Year']==1970]['fertility'],
                     'y':data.loc[data['Year']==1970]['life'],
                     'country': data.loc[data['Year']==1970]['Country'],
                     'pop': (data.loc[data['Year']==1970]['population']/ 20000000) + 2,
                     'region': data.loc[data['Year']==1970]['region']}
                )
xmin, xmax = min(data.fertility), max(data.fertility)
ymin, ymax = min(data.life), max(data.life)

#color mapping
regions_list = data.region.unique().tolist()

from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)
hover = HoverTool(tooltips=[('Country', '@country')])

In [51]:
#plot
p= figure(title='Interactive Dashboard', 
          x_axis_label='Fertility', y_axis_label='life expectancy')
p.circle(x='x', y='y', fill_alpha=0.8, source=source, color=dict(field='region', transform=color_mapper), legend='region')
p.legend.location='bottom_right'
p.add_tools(hover)

In [52]:

# Define the callback function: update_plot
def update_plot(attr, old, new):
    # Set the yr name to slider.value and new_data to source.data
    yr = slider.value
    x = x_select.value
    y = y_select.value
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    print(x)
    
    new_data = {
        'x'       : data.loc[data['Year']==yr][x],
        'y'       : data.loc[data['Year']==yr][y],
        'country' : data.loc[data['Year']==yr].Country,
        'pop'     : (data.loc[data['Year']==yr].population / 20000000) + 2,
        'region'  : data.loc[data['Year']==yr].region
    }
    source.data = new_data
    p.title.text = 'Gapminder data for %d' % yr
    # Set the range of all axes
    p.x_range.start = min(data[x])
    p.x_range.end = max(data[x])
    p.y_range.start = min(data[y])
    p.y_range.end = max(data[y])
    
# Make a slider object: slider
slider = Slider(start=1970, end=2010, step=1, value=1970, title='Year')

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)

#dropdown
x_select = Select(
    options=['fertility', 'life', 'child_mortality', 'gdp'],
    value='fertility',
    title='x-axis data'
)
x_select.on_change('value', update_plot)

y_select = Select(
    options=['fertility', 'life', 'child_mortality', 'gdp'],
    value='life',
    title='y-axis data'
)

y_select.on_change('value', update_plot)

# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider,x_select, y_select), p)
curdoc().add_root(layout)
curdoc().title = 'Gapminder'

In [53]:
output_file('gapminder.html')
show(p)